In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.calcium import calcium

from utils.wheel import wheel

Autosaving every 180 seconds


In [2]:
#########################################################
########## COMPUTE VELOCITY FROM ROTARY ENCODER ##########
##########################################################
from tqdm import trange

#
def binarize(data):
    
    idx = np.where(data<2.5)[0]
    idx2 = np.where(data>=2.5)[0]
    
    data[idx] = 0
    data[idx2] = 1
    
    #
    return data

#
def get_velocity(rot1, rot2):
    
    # distance
    n_clicks_per_rotation = 500
    ball_diameter = 0.2  # distance in meters
    ball_circumference = ball_diameter*3.141592
    dist_per_click = ball_circumference/n_clicks_per_rotation
    
    # time
    sample_rate = 1000
    seconds_per_time_stamp = 1/sample_rate
    
    
    #
    bin1 = binarize(rot1)
    bin2 = binarize(rot2)
    
    #
    clicks = np.array((bin1, bin2)).T.squeeze()
    print (clicks)
        
    #
    vel = []
    times = []
    last_click_location = 0
    time_since_last_click = 0
    rot1_last_state = clicks[0,0]
    rot2_state = clicks[0,1]
    for k in trange(0,clicks.shape[0],1):
        
        #
        if clicks[k,0]!=rot1_last_state:
            distance = dist_per_click  #only walked 1 click
            time = time_since_last_click*seconds_per_time_stamp
            
            # 
            v = distance/time 
            
            #
            vel.append(v)
            times.append(k/sample_rate)       
            
            #
            time_since_last_click=0
            last_click_location=k #.copy()
            rot1_last_state=clicks[k,0]
        else:
            vel.append(0)
            times.append(k/sample_rate)       

        #    
        time_since_last_click+=1
    #
    return np.array(vel), np.array(times)
    

In [10]:
#
# import scipy
# fname_mat = '/media/cat/4TB1/donato/DON-009191/20220220/TRD-2P/DON-009191_20220220_TRD-2P_S1-ACQ.mat'

# data = scipy.io.loadmat(fname_mat)
# #print (data)

# print (data['trainingdata'].shape)
# rot1 = data['trainingdata'][:,0]
# rot2 = data['trainingdata'][:,1]

# #
# vel, times = get_velocity(rot1,rot2)

root_dir = '/run/user/1000/gvfs/smb-share:server=scicore-toucan.scicore.unibas.ch,share=donafl00-calcium$/Users/Rodrigo/DON-002865/'

fnames = [
'20210210/TRD-2P/wheel.npy',
'20210211/TRD-2P/wheel.npy',
'202102112/TRD-2P/wheel.npy',
'202102113/TRD-2P/wheel.npy',
'202102114/TRD-2P/wheel.npy',
'202102115/TRD-2P/wheel.npy',
'202102116/TRD-2P/wheel.npy',
'202102117/TRD-2P/wheel.npy',
'202102118/TRD-2P/wheel.npy',
'202102119/TRD-2P/wheel.npy', 
]

#
for fname in fnames:
    fname1 = os.path.join(root_dir, fname)
    print (fname1)
    #
    data_wheel = np.load(fname1)

    w = wheel.Wheel()
    w.root_dir = os.path.split(fname1)[0]
    w.load_track()
    w.compute_velocity()
                               
                            
    #break


/run/user/1000/gvfs/smb-share:server=scicore-toucan.scicore.unibas.ch,share=donafl00-calcium$/Users/Rodrigo/DON-002865/20210210/TRD-2P/wheel.npy
/run/user/1000/gvfs/smb-share:server=scicore-toucan.scicore.unibas.ch,share=donafl00-calcium$/Users/Rodrigo/DON-002865/20210211/TRD-2P/wheel.npy


100%|█████████████████████████████| 2787774/2787774 [00:12<00:00, 214888.28it/s]


non-zero rotary encoder vals:  [      26       54       83 ... 18299922 18299956 18299990]


100%|███████████████████████████████| 332903/332903 [00:02<00:00, 122175.37it/s]


vel_times:  [      26       54       83 ... 18299922 18299956 18299990]


...

non-zero rotary encoder vals:  [    23     56     90 ... 899934 899962 899991]


100%|█████████████████████████████████| 27218/27218 [00:00<00:00, 120962.27it/s]


vel_times:  [    23     56     90 ... 899933 899962 899991]


refilling stationary times: 100%|█████| 27217/27217 [00:00<00:00, 196287.29it/s]


vel times:  (27218,)
vel values:  (27218,) [0.54636395 0.3807991  0.36959913 ... 0.43332312 0.44879895 0.43332312]
self.track.galvo_triggers.times:  [ 68692  69015  69338 ... 899322 899645 899968]
vel_times: [    23     56     90 ... 899933 899962 899991]


In [10]:
####################################################
def get_h5py_speed(filename):
    import h5py
    #filename = "file.hdf5"

    with h5py.File(filename, "r") as f:
        # Print all root level object names (aka keys) 
        # these can be group or dataset names 
        print("Keys: %s" % f.keys())
        # get first object name/key; may or may NOT be a group
        a_group_key = list(f.keys())[1]

        # get the object type for a_group_key: usually group or dataset
        print(type(f[a_group_key])) 

        # If a_group_key is a group name, 
        # this gets the object names in the group and returns as a list
        data = list(f[a_group_key])

        # If a_group_key is a dataset name, 
        # this gets the dataset values and returns as a list
        data = list(f[a_group_key])

        # preferred methods to get dataset values:
        ds_obj = f[a_group_key]      # returns as a h5py dataset object
        ds_arr = f[a_group_key][()].squeeze()  # returns as a numpy array

        #

    return ds_arr

################################################
vel_python = np.load('/media/cat/4TB1/donato/DON-009191/20220217/TRD-2P/velocity.npy')*100


filename = '/media/cat/4TB1/donato/DON-009191/20220217/TRD-2P/trd_var.mat'
vel_matlab = get_h5py_speed(filename)
####################################################

plt.figure()
#times = np.arange(vel_python.shape[0])/30.
plt.plot(w.track.velocity.times[start:end],
         w.track.velocity.values[start:end]*100, c='blue', label='python')

#
times_matlab = np.arange(vel_matlab.shape[0])/10000.
plt.plot(times_matlab[::30], vel_matlab[::30], c='red',label='matlab')
plt.legend(fontsize=16)
plt.show()

Keys: <KeysViewHDF5 ['raw_dist', 'spd', 'tt']>
<class 'h5py._hl.dataset.Dataset'>


In [5]:
##################################################################
############# COMPUTE VELOCITY FROM ROTARY ENCODERS ##############
##################################################################
data = np.load('/home/cat/Downloads/Downloads/results.npz',
               allow_pickle=True)

#
lick_detector_times = data['lick_detector_abstime']
print ("lick detector: ", lick_detector_times.shape, lick_detector_times)

#
reward_times = data['reward_times'].squeeze().T[:,1]
#print ("reward times: ", reward_times[:50])
ttl_times = data['ttl_times']
#print ("ttl times: ", ttl_times)

#
rot1 = data['rotary_encoder1_abstime']
rot2 = data['rotary_encoder2_abstime']
#print ("rotary 1: ", rot1)
#print ("rotary 2: ", rot2)

#
vel, times = get_velocity(rot1,rot2)

plt.figure()
plt.plot(times, vel)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/home/cat/Downloads/Downloads/results.npz'

In [21]:
#
vel2 = np.load('/media/cat/4TB1/donato/DON-009191/20220217/TRD-2P/wheel.npy').squeeze()
print (vel.shape, vel2.shape)

plt.figure()
#plt.plot(times, vel, c='blue')
plt.plot(times, vel2, c='red',)
plt.show()

(18300000,) (18300000,)


In [67]:
#################################################
#################### PUP IDS ####################
#################################################

#
mouse_ids = [
    #'006084',
    #'006085',
    #'006087',
    '009191',
    '009192',
    '010473',
    '010477'
]

#
PDays = [
    #[19,28],
    #[19,28],
    #[24,33],
    [16,48],  # expression issues
    [15,60],  
    [16,33],
    [15,33],
]

#
fig=plt.figure()
ax=plt.subplot()
xlabels = mouse_ids

#
for k in range(len(mouse_ids)):
    #y = np.arange(PDays[k][0],PDays[k][1],0.1)
    x = np.arange(PDays[k][0],PDays[k][1],0.1)
    y = k
    #plt.fill_betweenx(y, k, k+0.9, 
                      #alpha=.5, label = mouse_ids[k])
    
    #x = np.arange(
    ax.fill_between(x,k,k+0.9,
                   label = mouse_ids[k])
    
    plt.text(np.median(x)-1,k+0.3,str(PDays[k][1]-PDays[k][0]+1))

#
plt.xlim(left=0)
plt.ylabel("mice ids")
plt.yticks(np.arange(len(mouse_ids))+0.5, 
           mouse_ids, 
           rotation=45)
plt.xlabel("P-Day")
plt.legend(ncol=2, fontsize=8, loc=4)
plt.show()




In [5]:
#################################################
################## ADULT IDS ####################
#################################################

#
mouse_ids = [
    '002865', 
    '003165',
    '003343',
    '008497',
    '008498',
    '008499'
]

#
PDays = [
    [90,99],
    [120,129],
    [150,159],
    [61,73],
    [61,73],
    [61,73],
]

#
fig=plt.figure()
ax=plt.subplot()
xlabels = mouse_ids

for k in range(len(mouse_ids)):
    #y = np.arange(PDays[k][0],PDays[k][1],0.1)
    x = np.arange(PDays[k][0],PDays[k][1],0.1)
    y = k
    #plt.fill_betweenx(y, k, k+0.9, 
                      #alpha=.5, label = mouse_ids[k])
    
    #x = np.arange(
    alpha=1.0
    if k >2:
        alpha=.3
    ax.fill_between(x,k,k+0.9, label = mouse_ids[k],
                   alpha=alpha)
    
    plt.text(np.median(x)-3,k+0.3,str(PDays[k][1]-PDays[k][0]+1))

plt.xlim(left=0)
plt.ylabel("mice ids")
plt.yticks(np.arange(len(mouse_ids))+0.5, mouse_ids,
          rotation=45)
plt.xlabel("P-Day")
plt.legend(ncol=2, fontsize=8, loc=4)
plt.show()


(18299999,)


(18299999,)
